In [48]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

trainData = pd.read_csv('/kaggle/input/digit-recognizer/train.csv')
testData = pd.read_csv('/kaggle/input/digit-recognizer/test.csv')

In [49]:
trainData.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [50]:
testData.head()

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Here, we see that the test dataset is only useful for the competition organizers, but we cannot use it due to the absence of a "label" column. To address this, I will split the training dataset into training and testing portions.

In [51]:
trainData = np.array(trainData)
rows, columns = trainData.shape
np.random.shuffle(data) # shuffle before splitting into dev and training sets

print(f"rows: {rows} , columns: {columns}")

rows: 42000 , columns: 785


In [52]:
testData = trainData[0:2000].T
testLabels = testData[0]
testPixels = testData[1:columns]
testPixels = testPixels / 255.

trainData = trainData[2000:rows].T
trainLabels = trainData[0]
trainPixels = trainData[1:columns]
trainPixels = trainPixels / 255.
_,rows_train = X_train.shape

In [53]:
print(testLabels)
print(f"shape : {testLabels.shape}")

[1 0 1 ... 5 5 4]
shape : (2000,)


In [54]:
print(testPixels)
print(f"shape: {testPixels.shape}")
print(f"shape: {testPixels[:,1].shape}")

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
shape: (784, 2000)
shape: (784,)


In [55]:
print(trainLabels)
print(f"shape : {trainLabels.shape}")

[3 1 6 ... 7 6 9]
shape : (40000,)


In [56]:
print(trainPixels)
print(f"shape: {trainPixels.shape}")
print(f"shape: {trainPixels[:,1].shape}")

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
shape: (783, 40000)
shape: (783,)


This all checks out so we can move on to building our network.

In [63]:
def init_params():
    W1 = np.random.rand(10, 783) - 0.5
    b1 = np.random.rand(10, 1) - 0.5
    W2 = np.random.rand(10, 10) - 0.5
    b2 = np.random.rand(10, 1) - 0.5
    return W1, b1, W2, b2

def ReLU(Z):
    return np.maximum(Z, 0)

def softmax(Z):
    A = np.exp(Z) / sum(np.exp(Z))
    return A
    
def forward_prop(W1, b1, W2, b2, X):
    Z1 = W1.dot(X) + b1
    A1 = ReLU(Z1)
    Z2 = W2.dot(A1) + b2
    A2 = softmax(Z2)
    return Z1, A1, Z2, A2

In [64]:
def ReLU_deriv(Z):
    return Z > 0

def one_hot(Y):
    one_hot_Y = np.zeros((Y.size, Y.max() + 1))
    one_hot_Y[np.arange(Y.size), Y] = 1
    one_hot_Y = one_hot_Y.T
    return one_hot_Y

def backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y):
    one_hot_Y = one_hot(Y)
    dZ2 = A2 - one_hot_Y
    dW2 = 1 / m * dZ2.dot(A1.T)
    db2 = 1 / m * np.sum(dZ2)
    dZ1 = W2.T.dot(dZ2) * ReLU_deriv(Z1)
    dW1 = 1 / m * dZ1.dot(X.T)
    db1 = 1 / m * np.sum(dZ1)
    return dW1, db1, dW2, db2

def update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha):
    W1 = W1 - alpha * dW1
    b1 = b1 - alpha * db1    
    W2 = W2 - alpha * dW2  
    b2 = b2 - alpha * db2    
    return W1, b1, W2, b2

In [65]:
def get_predictions(A2):
    return np.argmax(A2, 0)

def get_accuracy(predictions, Y):
    print(predictions, Y)
    return np.sum(predictions == Y) / Y.size

def gradient_descent(X, Y, alpha, iterations):
    W1, b1, W2, b2 = init_params()
    for i in range(iterations):
        Z1, A1, Z2, A2 = forward_prop(W1, b1, W2, b2, X)
        dW1, db1, dW2, db2 = backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y)
        W1, b1, W2, b2 = update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha)
        if i % 10 == 0:
            print("Iteration: ", i)
            predictions = get_predictions(A2)
            print(get_accuracy(predictions, Y))
    return W1, b1, W2, b2

In [66]:
W1, b1, W2, b2 = gradient_descent(trainPixels, trainLabels, 0.10, 500)

Iteration:  0
[2 2 2 ... 2 2 2] [3 1 6 ... 7 6 9]
0.086425
Iteration:  10
[8 3 2 ... 7 3 2] [3 1 6 ... 7 6 9]
0.172625
Iteration:  20
[8 3 4 ... 7 3 3] [3 1 6 ... 7 6 9]
0.244825
Iteration:  30
[8 3 4 ... 7 5 3] [3 1 6 ... 7 6 9]
0.319325
Iteration:  40
[8 3 4 ... 7 6 8] [3 1 6 ... 7 6 9]
0.398025
Iteration:  50
[8 8 6 ... 7 6 8] [3 1 6 ... 7 6 9]
0.473625
Iteration:  60
[8 8 6 ... 7 6 8] [3 1 6 ... 7 6 9]
0.525275
Iteration:  70
[8 8 6 ... 7 6 8] [3 1 6 ... 7 6 9]
0.571
Iteration:  80
[8 8 6 ... 7 6 8] [3 1 6 ... 7 6 9]
0.6089
Iteration:  90
[8 1 6 ... 7 6 8] [3 1 6 ... 7 6 9]
0.642825
Iteration:  100
[8 1 6 ... 7 6 9] [3 1 6 ... 7 6 9]
0.67125
Iteration:  110
[8 1 6 ... 7 6 9] [3 1 6 ... 7 6 9]
0.692275
Iteration:  120
[8 1 6 ... 7 6 9] [3 1 6 ... 7 6 9]
0.7105
Iteration:  130
[8 1 6 ... 7 6 9] [3 1 6 ... 7 6 9]
0.7265
Iteration:  140
[8 1 6 ... 7 6 9] [3 1 6 ... 7 6 9]
0.738575
Iteration:  150
[8 1 6 ... 7 6 9] [3 1 6 ... 7 6 9]
0.74965
Iteration:  160
[8 1 6 ... 7 6 9] [3 1 6 ... 7

In [67]:
def make_predictions(X, W1, b1, W2, b2):
    _, _, _, A2 = forward_prop(W1, b1, W2, b2, X)
    predictions = get_predictions(A2)
    return predictions

def test_prediction(index, W1, b1, W2, b2):
    current_image = X_train[:, index, None]
    prediction = make_predictions(X_train[:, index, None], W1, b1, W2, b2)
    label = Y_train[index]
    print("Prediction: ", prediction)
    print("Label: ", label)
    
    current_image = current_image.reshape((28, 28)) * 255
    plt.gray()
    plt.imshow(current_image, interpolation='nearest')
    plt.show()

In [68]:
test_prediction(0, W1, b1, W2, b2)
test_prediction(1, W1, b1, W2, b2)
test_prediction(2, W1, b1, W2, b2)
test_prediction(3, W1, b1, W2, b2)

Prediction:  [9]
Label:  0


ValueError: cannot reshape array of size 783 into shape (28,28)